In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import math
import numpy as np
from numba import cuda
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from matplotlib.pyplot import figure

from keras.layers import Dropout

from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

from tqdm.notebook import tqdm

#Importing the Libraries
%matplotlib inline
import matplotlib. pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.dates as mandates
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import adam_v2, adamax_v2, adadelta_v2, adagrad_v2, rmsprop_v2
from keras.models import load_model
from keras.layers import LSTM, GRU
from keras.utils.vis_utils import plot_model
import keras
import pickle

from statsmodels.tsa.stattools import grangercausalitytests

In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# **Read Dataset**

In [ ]:
df = pd.read_csv(path+filename, parse_dates=['timestamp'])
df = df[df.timestamp > '2020-12-31 00:00:00']
#df = df[df.month == 1]
#df = df[df.day <= 4]
#df = df[df.time <= '12:00:00']
df.reset_index(inplace=True)
del df['index']
del df['ignore']
del df['close_time']
del df['quote_av']
del df['tb_quote_av']
del df['tb_base_av']
df.timestamp = pd.to_datetime(df.timestamp)
df.set_index('timestamp')
df

In [ ]:
df["open"].plot()

In [ ]:
df_test = pd.read_csv(path+filename, parse_dates=['timestamp'])
df_test = df_test[df_test.timestamp >= '2021-12-31 00:00:00']
#df_test = df_test[df_test.month == 1]
#df_test = df_test[df_test.day <= 4]
#df_test = df_test[df_test.time <= '12:00:00']
df_test.reset_index(inplace=True)
del df_test['index']
df_test.timestamp = pd.to_datetime(df_test.timestamp)
df_test.set_index('timestamp')
df_test

In [ ]:
df_test["open"].plot()

# **Prepro**

## **Get percentage change**

In [ ]:
df_test['pct_change'] = df_test['open'].pct_change()

df['pct_change'] = df['open'].pct_change()
df

## **Differential**

In [ ]:
# diff 1
df_test['diff_1'] = df_test['open'].diff()
df_test['diff_2'] = df_test['open'].diff().diff()
df_test

df['diff_1'] = df['open'].diff()
df['diff_2'] = df['open'].diff().diff()
df

## **Detrend**

In [ ]:
from scipy import signal

df_test['detrend'] = signal.detrend(df_test['open'].values)
df_test

df['detrend'] = signal.detrend(df['open'].values)
df

## **Smoothing**

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess

df_test['MA'] = df_test['open'].rolling(5, center=True).mean()
#df_test['loess_5'] = pd.DataFrame(lowess(df_test['open'], np.arange(len(df_test['open'])), frac=0.05)[:, 1], index=df_test['open'], columns=['open'])
#df_test['loess_15'] = pd.DataFrame(lowess(df_test['open'], np.arange(len(df_test['open'])), frac=0.15)[:, 1], index=df_test['open'], columns=['open'])

df['MA'] = df['open'].rolling(3, center=True).mean()
#df['loess_5'] = pd.DataFrame(lowess(df['open'], np.arange(len(df['open'])), frac=0.05)[:, 1], index=df['open'], columns=['open'])
#df['loess_15'] = pd.DataFrame(lowess(df['open'], np.arange(len(df['open'])), frac=0.15)[:, 1], index=df['open'], columns=['open'])
df

## **Get profit status**

In [ ]:
df_test['profit'] = df_test['pct_change'].apply(lambda x: 1000 if x > 0.0016 else 0)

df['profit'] = df['pct_change'].apply(lambda x: 1000 if x > 0.0016 else 0)

## **Drop NaN**

In [ ]:
df_test.dropna(inplace=True)
#df_test['pct_change'] = df_test['pct_change'] + 2
#df_test['diff_1'] = df_test['diff_1'] + 2000
#df_test['diff_2'] = df_test['diff_2'] + 2000
#df_test['detrend'] = df_test['detrend'] + 2000
df_test.reset_index(inplace=True)
del df_test['index']

df.dropna(inplace=True)
#df['pct_change'] = df['pct_change'] + 2
#df['diff_1'] = df['diff_1'] + 2000
#df['diff_2'] = df['diff_2'] + 2000
#df['detrend'] = df['detrend'] + 2000
df.reset_index(inplace=True)
del df['index']

df

## **Set target**

In [ ]:
#Set Target Variable
target = 'close'
y_test = df_test[target]
y_train = df[target]

## **Set features**

### **Get features correlation to target using Granger**

In [ ]:
df_ = df.copy()
del df_['timestamp']

In [ ]:
maxlag=16
test='ssr_chi2test'
result = []

for c in tqdm(df_.columns):
  if c != target:
    test_result = grangercausalitytests(df_[[target, c]], maxlag=maxlag, verbose=False)
    p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
    min_p_value = np.min(p_values)
    result.append([c, min_p_value, p_values.index(min_p_value)+1])

granger = pd.DataFrame(result, columns=['feature', target, 'lag'])
granger.set_index('feature')
granger

In [ ]:
features = []
for i, f in granger.iterrows():
  if f[target] <0.005:
    features.append(f['feature'])
features

## **Set train test data**

In [ ]:
#from tensorflow.keras.utils import to_categorical
#y_test = to_categorical(y_test)
#y_train = to_categorical(y_train)

In [ ]:
trainX =  np.array(df[features])
testX =  np.array(df_test[features])

X_train = trainX.reshape(df[features].shape[0], 1, df[features].shape[1])
X_test = testX.reshape(df_test[features].shape[0], 1, df_test[features].shape[1])

# **Model**

In [ ]:
tf.get_logger().setLevel('ERROR')

In [ ]:
lstm = Sequential()
lstm.add(LSTM(1210, input_shape=(1, trainX.shape[1]), use_bias=True, return_sequences=True, activation = "elu", bias_regularizer='l2'))
#lstm.add(Dropout(0.1))
lstm.add(LSTM(990, use_bias=True, return_sequences=True, activation = "elu", bias_regularizer='l2'))
#lstm.add(Dropout(0.1))
#lstm.add(LSTM(1100, return_sequences=True, use_bias=True, activation = "elu"))
#lstm.add(Dense(1100, use_bias=True, activation = "elu"))

lstm.add(Dense(770, use_bias=True, activation = "elu", bias_regularizer='l2'))
#lstm.add(Dense(330, use_bias=True, activation = "elu"))
lstm.add(Dense(99, use_bias=True, activation = "elu", bias_regularizer='l2'))
lstm.add(Dense(11, use_bias=True, activation = "elu", bias_regularizer='l2'))
lstm.add(Dense(1, activation = "elu"))

lstm.compile(loss="mean_absolute_error", optimizer="rmsprop")
#plot_model(lstm, show_shapes=True, show_layer_names=True)

## **Train**

In [ ]:
epoch = 100
bs = 64
patience_lr = 15
patience_es = patience_lr * 3

In [ ]:
filepath = path+"weight/Weight_ETHBUSD_5m.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=patience_lr, verbose=1, min_delta=1e-3, mode='auto')
early_stop = EarlyStopping(monitor='val_loss', patience=patience_es, verbose=1, mode='auto')
callbacks_list = [checkpoint, reduce_lr, early_stop]
    
history = lstm.fit(X_train, y_train, 
                    epochs=epoch, batch_size=bs, 
                    validation_data=(X_test[:], y_test[:]), 
                    verbose=1, shuffle=False, callbacks=callbacks_list)
                   

## **Plot**

In [ ]:
plt.plot(history.history['loss'],'b',label='loss')
plt.plot(history.history['val_loss'],'r', label='val_loss')
plt.legend(loc="best")
plt.title('MODEL VAL LOSS')

## **Eval**

In [ ]:
lstm = load_model(filepath)
y_pred = lstm.predict(X_test)

y_predd = []
for y in y_pred:
  y_predd.append(y[0])

In [ ]:
plt.plot(y_predd[:], 'r', label='pred')
plt.plot(y_test[:], 'b', label='actual')
plt.legend(loc="best")

In [ ]:
df_eval = pd.DataFrame(y_test)
df_eval
df_eval ['pct_change_actual'] = df_eval ['open'].pct_change()
df_eval['profit_actual'] = df_eval['pct_change_actual'].apply(lambda x: 1000 if x > 0.0016 else 0)

df_eval ['open_pred'] = y_predd
df_eval['pct_change'] = df_eval['open_pred'].pct_change()
df_eval['profit'] = df_eval['pct_change'].apply(lambda x: 1000 if x > 0.0016 else 0)

df_eval

## **Accuracy profit**

In [ ]:
from sklearn.metrics import classification_report
print( classification_report(df_eval['profit_actual'], df_eval['profit']))